In [1]:
import os 
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset


def load_dataset(path):
    def get_split_dataset(mode):
        dataset = {}
        for f in ["molecule", "protein", "y"]:
            f_path = os.path.join(path, mode + "_" + f + ".npy")
            print(f_path)
            data = np.load(f_path, allow_pickle=True)
            try:
                data = torch.tensor([d.squeeze(0).numpy() for d in data])
            except:
                data = torch.tensor(data)
            dataset[f] = data
            
        return dataset
            
    train_data = get_split_dataset("train")
    valid_data = get_split_dataset("valid")
    test_data = get_split_dataset("test")
    
    return train_data, valid_data, test_data
    
train_data, valid_data, test_data = load_dataset("data/interaction/kiba")

data/interaction/kiba/train_molecule.npy


/tmp/ipykernel_12625/1447363121.py:19: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180595841/work/torch/csrc/utils/tensor_new.cpp:201.)
  data = torch.tensor([d.squeeze(0).numpy() for d in data])


data/interaction/kiba/train_protein.npy
data/interaction/kiba/train_y.npy
data/interaction/kiba/valid_molecule.npy
data/interaction/kiba/valid_protein.npy
data/interaction/kiba/valid_y.npy
data/interaction/kiba/test_molecule.npy
data/interaction/kiba/test_protein.npy
data/interaction/kiba/test_y.npy


In [2]:
train_dataset = TensorDataset(train_data['molecule'], train_data['protein'], train_data['y'])
valid_dataset = TensorDataset(valid_data['molecule'], valid_data['protein'], valid_data['y'])
test_dataset = TensorDataset(test_data['molecule'], test_data['protein'], test_data['y'])
    
train_dataloader = DataLoader(train_dataset, batch_size=512, num_workers=16, 
                              shuffle=True, pin_memory=True, prefetch_factor=10, 
                              drop_last=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=512, num_workers=16, 
                              shuffle=False, pin_memory=True, prefetch_factor=10, 
                              drop_last=False)
test_dataloader = DataLoader(test_dataset, batch_size=512, num_workers=16, 
                             shuffle=False, pin_memory=True, prefetch_factor=10, 
                             drop_last=False)

In [3]:
class ConcatenateDTI(nn.Module):
    def __init__(self, molecule_dim=128, protein_dim=1024, inner_dim=512, projection=True):
        super().__init__()
        self.is_projection = projection

        if self.is_projection:
            self.mol_proj = nn.Linear(molecule_dim, inner_dim)        
            self.prot_proj = nn.Linear(protein_dim, inner_dim)            
            self.fc_1 = nn.Linear(inner_dim * 2, inner_dim)
        else:
            self.fc_1 = nn.Linear(molecule_dim + protein_dim, inner_dim)
        
        self.fc_2 = nn.Linear(inner_dim, int(inner_dim / 2))
        self.fc_out = nn.Linear(int(inner_dim / 2), 1)
   

    def forward(self, molecule, protein):
        if self.is_projection:
            molecule = self.mol_proj(molecule)
            protein = self.prot_proj(protein)
            
        x = torch.cat((molecule, protein), -1)
        x = F.dropout(F.gelu(self.fc_1(x)), 0.1)
        x = F.dropout(F.gelu(self.fc_2(x)), 0.1)
        x = self.fc_out(x)
        
        return x
        
        
concatenate_dti = ConcatenateDTI()
concatenate_dti

ConcatenateDTI(
  (mol_proj): Linear(in_features=128, out_features=512, bias=True)
  (prot_proj): Linear(in_features=1024, out_features=512, bias=True)
  (fc_1): Linear(in_features=1024, out_features=512, bias=True)
  (fc_2): Linear(in_features=512, out_features=256, bias=True)
  (fc_out): Linear(in_features=256, out_features=1, bias=True)
)

In [ ]:
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.trainer.supporters import CombinedLoader


class DTI_prediction(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
        self.train_accuracy = torchmetrics.MeanAbsoluteError()
        self.valid_accuracy = torchmetrics.MeanAbsoluteError()
        self.test_accuracy = torchmetrics.MeanAbsoluteError()
        
        
    def forward(self, molecule, protein):
        return self.model(molecule, protein)
   
    
    def training_step(self, batch, batch_idx):
        molecule = batch[0]
        protein = batch[1]
        y = batch[2]
        
        y_hat = self(molecule, protein).squeeze(-1)        
        loss = F.mse_loss(y_hat, y)
        
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_accuracy", self.train_accuracy(y_hat, y), on_step=False, on_epoch=True, prog_bar=True)
        
        return loss

        
    def valid_step(self, batch, batch_idx):
        molecule = batch[0]
        protein = batch[1]
        y = batch[2]
        
        y_hat = self(molecule, protein).squeeze(-1)        
        loss = F.mse_loss(y_hat, y)
        
        self.log('valid_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("valid_accuracy", self.valid_accuracy(y_hat, y), on_step=False, on_epoch=True, prog_bar=True)
    
    
    def test_step(self, batch, batch_idx):
        molecule = batch[0]
        protein = batch[1]
        y = batch[2]
        
        y_hat = self(molecule, protein).squeeze(-1)        
        loss = F.mse_loss(y_hat, y)
        
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_accuracy", self.test_accuracy(y_hat, y), on_step=False, on_epoch=True, prog_bar=True)
    
    
    def predict_step(self, batch, batch_idx):
        molecule = batch[0]
        protein = batch[1]
        y = batch[2]
        
        y_hat = self(molecule, protein).squeeze(-1)        
        
        return y_hat
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=100)
    
        return {"optimizer": optimizer, "lr_scheduler": scheduler}
    
    
callbacks = [
    ModelCheckpoint(monitor='valid_loss', save_top_k=20, dirpath='weights/DTI_prediction_CLS_token_concatenate', filename='attentional_dti-{epoch:03d}-{valid_loss:.4f}-{valid_accuracy:.4f}'),
]

model = DTI_prediction(
    model=concatenate_dti
)

# remove precision 16, because prot bert trained using full precision
trainer = pl.Trainer(max_epochs=1000, gpus=1, enable_progress_bar=True, callbacks=callbacks)

In [ ]:
trainer.fit(model)

In [ ]:
# checkpoint_file = ""
# model.load_from_checkpoint(model=concatenate_dti, checkpoint_path="weights/DTI_prediction_CLS_token_concatenate/" + checkpoint_file)

# trainer.test(model, test_dataloader)